In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pickle
import h5py
from misc import *
import plotly.express as px
import pandas as pd 

from sklearn.decomposition import PCA
import phate
from umap import UMAP
from sklearn.manifold import TSNE

## 1. Import/generate Data

In [ ]:
"""load saved trajectories data for npz file
"""
# SEQ = "PT3"
SEQ = "PT4_hairpin"

# multiple trajectories
if SEQ in ["PT3", "PT4", "PT3_hairpin"]:
     fnpz_data = "data/helix_assoc/helix_assoc_{}_multrj_100epoch_py.npz".format(SEQ)
elif SEQ in ["PT0", "PT4_hairpin"]:
     fnpz_data = "data/helix_assoc/helix_assoc_{}_multrj_60epoch_py.npz".format(SEQ)

data_npz = np.load(fnpz_data)

# asssign data to variables
for var in data_npz.files:
     locals()[var] = data_npz[var]

# recover full data based on coord_id, indices, and unique data
SIMS_adj = SIMS_adj_uniq[coord_id_S]
SIMS_scar = SIMS_scar_uniq[coord_id_S]
SIMS_G = SIMS_G_uniq[coord_id_S]
SIMS_pair = SIMS_pair_uniq[coord_id_S]

print(SIMS_T.shape,SIMS_HT.shape,SIMS_HT_uniq.shape)
print(SIMS_adj.shape,SIMS_scar.shape,SIMS_G.shape,SIMS_HT.shape,SIMS_pair.shape)
print(SIMS_adj_uniq.shape,SIMS_scar_uniq.shape,SIMS_G_uniq.shape,SIMS_pair_uniq.shape) 
print(SIMS_dict.shape,SIMS_dict_uniq.shape)
print(coord_id_S.shape,indices_S.shape,trj_id.shape,data_embed.shape,occ_density_S.shape)
print(pca_coords.shape,pca_all_coords.shape)
print(phate_coords.shape,phate_all_coords.shape)
print(umap_coord_2d.shape,umap_all_coord_2d.shape,umap_coord_3d.shape,umap_all_coord_3d.shape)
print(tsne_coord_2d.shape,tsne_all_coord_2d.shape,tsne_coord_3d.shape,tsne_all_coord_3d.shape)

### 1.1 Load multiple simulated trajectory from Mulistrand

In [ ]:
# load multiple trajectories from multiple files
SEQ = "PT4_hairpin"
# SEQ = "PT4"

folder_name = "data/helix_assoc_{}/assoc_{}_1sim_20C".format(SEQ,SEQ)

# define absorbing (final) state structure
FINAL_STRUCTURE = "(((((((((((((((((((((((((+)))))))))))))))))))))))))"
num_files = 100

SIMS,SIMS_retrieve,SIMS_concat = load_multitrj(folder_name,FINAL_STRUCTURE,num_files)

print("SIMS: ", len(SIMS))
print("SIMS_retrieve: ", SIMS_retrieve.shape)
print("SIMS_concat: ", len(SIMS_concat))

### 2. Convert dot-paren to adjacency matrix

In [ ]:
""" Dimenstions of SIM_adj list 
SIM_adj: N*m*m
    N: number of states in the trajectory
    m: number of nucleotides in the state (strand)
"""
# get multiple trajectories' data
SIMS_adj, SIMS_G, SIMS_T, SIMS_HT, SIMS_pair, trj_id = sim_adj(SIMS_concat)
print(SIMS_adj.shape,SIMS_G.shape,SIMS_T.shape,SIMS_HT.shape,SIMS_pair.shape,trj_id.shape)

### 3.1 Get unique data except holding time

In [ ]:
# get unique states adjacency matrix with their occupancy density, get unique energy, time, if paired;
# and their corresponding indices

# multiple trajectories
indices_S,occ_density_S,SIMS_adj_uniq,SIMS_G_uniq,SIMS_pair_uniq \
     = get_unique(SIMS_concat,SIMS_adj,SIMS_G,SIMS_pair) 
print(indices_S.shape, occ_density_S.shape, SIMS_adj_uniq.shape,SIMS_G_uniq.shape,SIMS_pair_uniq.shape)

### 3.2. Get labeled trajectory data

In [ ]:
# # get trajectory data with its corresponding labels 
# # multiple trajectories
SIMS_dict = label_structures(SIMS,indices_S)
coord_id_S = SIMS_dict[:,-1].astype(int)
SIMS_dict_uniq = np.array(SIMS)[indices_S]
print(SIMS_dict.shape, coord_id_S.shape, SIMS_dict_uniq.shape)

# find the structure having the largest occupancy density
print(SIMS_retrieve[indices_S[occ_density_S.argmax()]])

### 3.3 Get unique holding time for each state

In [ ]:
# get unique holding time of unique states
SIMS_HT_uniq = mean_holdingtime(SIMS_HT, indices_S, coord_id_S)
print(SIMS_HT_uniq.shape)

### 4. Convert adjacency matrix scattering coefficients

#### SIMS_scar_uniq

In [ ]:
# # Multiple trajectories
scat_coeff_array_S = transform_dataset(SIMS_adj_uniq)
SIMS_scar_uniq = get_normalized_moments(scat_coeff_array_S).squeeze()

# get SIMS_scar based on SIMS_scar_uniq
SIMS_scar = SIMS_scar_uniq[coord_id_S]

In [ ]:
# print(SIMS_scar.shape, (np.unique(SIMS_scar,axis=0)).shape)
print(SIMS_scar_uniq.shape, (np.unique(SIMS_scar_uniq,axis=0)).shape)

In [ ]:
# # For large trajectories states
# SIMS_scar_uniq1 = get_normalized_moments(transform_dataset(SIMS_adj_uniq[:60000])).squeeze()
# SIMS_scar_uniq2 = get_normalized_moments(transform_dataset(SIMS_adj_uniq[60000:])).squeeze()
# SIMS_scar_uniq = np.concatenate((SIMS_scar_uniq1,SIMS_scar_uniq2))

# # get SIMS_scar based on SIMS_scar_uniq
# SIMS_scar = SIMS_scar_uniq[coord_id_S]

# print(SIMS_scar.shape, (np.unique(SIMS_scar,axis=0)).shape)
# print(SIMS_scar_uniq.shape, (np.unique(SIMS_scar_uniq,axis=0)).shape)

In [ ]:
""" Save all obtained data to npz file for python,
    Multiple trajectories
"""
# # save for python
# fnpz_data = "data/helix_assoc/helix_assoc_{}_multrj_100epoch_py_temp.npz".format(SEQ)
# with open(fnpz_data, 'wb') as f:
#     np.savez(f,
#             # SIMS data
#             SIMS_T=SIMS_T, SIMS_HT=SIMS_HT, SIMS_HT_uniq=SIMS_HT_uniq,
#             SIMS_adj_uniq=SIMS_adj_uniq, SIMS_scar_uniq=SIMS_scar_uniq,
#             SIMS_G_uniq=SIMS_G_uniq, SIMS_pair_uniq=SIMS_pair_uniq,
#             SIMS_dict=SIMS_dict, SIMS_dict_uniq=SIMS_dict_uniq,
#             # Indices
#             coord_id_S=coord_id_S, indices_S=indices_S,trj_id=trj_id, occ_density_S=occ_density_S,
#             # # embed data and occpancy density
#             # data_embed=data_embed,
#             # # plotting data
#             # pca_coords=pca_coords, pca_all_coords=pca_all_coords,
#             # phate_coords=phate_coords, phate_all_coords=phate_all_coords,
#             # umap_coord_2d=umap_coord_2d, umap_all_coord_2d=umap_all_coord_2d,
#             # umap_coord_3d=umap_coord_3d, umap_all_coord_3d=umap_all_coord_3d,
#             # tsne_coord_2d=tsne_coord_2d, tsne_all_coord_2d=tsne_all_coord_2d,
#             # tsne_coord_3d=tsne_coord_3d, tsne_all_coord_3d=tsne_all_coord_3d,
#             )
    
# # multiple trajectories
# fnpz_data = "data/helix_assoc/helix_assoc_PT4_multrj_100epoch_py_temp.npz"
# data_npz = np.load(fnpz_data)

# # asssign data to variables
# for var in data_npz.files:
#      locals()[var] = data_npz[var]

# # recover full data based on coord_id, indices, and unique data
# SIMS_adj = SIMS_adj_uniq[coord_id_S]
# SIMS_scar = SIMS_scar_uniq[coord_id_S]
# SIMS_G = SIMS_G_uniq[coord_id_S]
# SIMS_pair = SIMS_pair_uniq[coord_id_S]


# print(SIMS_T.shape,SIMS_HT.shape,SIMS_HT_uniq.shape)
# print(SIMS_adj.shape,SIMS_scar.shape,SIMS_G.shape,SIMS_HT.shape,SIMS_pair.shape)
# print(SIMS_adj_uniq.shape,SIMS_scar_uniq.shape,SIMS_G_uniq.shape,SIMS_pair_uniq.shape) 
# print(SIMS_dict.shape,SIMS_dict_uniq.shape)
# print(coord_id_S.shape,indices_S.shape,trj_id.shape,occ_density_S.shape)

### 5. Split data into tranning and test sets

In [ ]:
"""Shape of split data
    train_data: [tr_adjs, tr_coeffs, tr_energies]
    test_data: [te_adjs, te_coeffs, te_energies]
"""
train_data,test_data = split_data(SIMS_adj_uniq,SIMS_scar_uniq,SIMS_G_uniq)  # multiple trj

### 6. Train and test dataloader

In [ ]:
"""Structure of train_tup when gnn=False
    train_tup: [train_coeffs,train_energy] 
"""
train_loader, train_tup, test_tup, valid_loader,early_stop_callback = load_trte(train_data,test_data,
                                              batch_size=64)
train_tup[0].shape, test_tup[0].shape, train_loader.batch_size

## 2.1 Load Model

In [ ]:
# set up hyperparameters
input_dim = train_tup[0].shape[-1]
len_epoch = len(train_loader)

hparams = {
    'input_dim':  input_dim,
    'bottle_dim': 25,
    'hidden_dim': 400, #not used in model
    
    'len_epoch': len_epoch,
    'learning_rate': 0.0001,
    'max_epochs': 30, # 60 for PT4_hairpin, PT0, 100 for others
    'n_gpus': 0,
    'batch_size': 64, #not used in model
    
    'alpha':1.0,
    'beta':0.0001,

}

hparams = argparse.Namespace(**hparams)

model = GSAE(hparams)
print(model)

## 2.2 Train Model

In [ ]:
trainer = pl.Trainer.from_argparse_args(hparams,
                                        max_epochs=hparams.max_epochs,
                                        gpus=hparams.n_gpus,
                                        # callbacks=[early_stop_callback],
                                        )
trainer.fit(model=model,
            train_dataloader=train_loader,
            val_dataloaders=valid_loader,)

In [ ]:
model

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/ --host localhost --port 8000
#  http://localhost:8000

In [ ]:
# # save the trained model
# fname_model = "models/helix_assoc_{}_multrj_model_{}epoch.pickle".format(SEQ,hparams.max_epochs) # multiple trj

# pickle.dump(model, open(fname_model, 'wb'))
# print('Trained model saved.')

## 3. Load Pretrained Models

In [ ]:
fname_model = "models/helix_assoc_{}_multrj_model_100epoch.pickle".format(SEQ)
model = pickle.load(open(fname_model, 'rb'))
model

## 4. Get Embeddings

In [ ]:
# # multiple trajectories
with torch.no_grad():
        data_embed = model.embed(torch.Tensor(SIMS_scar_uniq))[0]

### 1. PCA

In [ ]:
# # do PCA for GSAE embeded data
pca_coords = PCA(n_components=3).fit_transform(data_embed)

# # get all pca embedded states coordinates
pca_all_coords = pca_coords[coord_id_S]  # multiple trj

pca_coords.shape, pca_all_coords.shape

In [ ]:
(np.unique(pca_coords,axis=0)).shape, (np.unique(pca_all_coords,axis=0)).shape

### 2. PHATE

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_embed = scaler.fit_transform(data_embed)
data_embed

In [ ]:
# # do PHATE for GSAE embeded data
phate_operator = phate.PHATE(n_jobs=-2)
phate_coords = phate_operator.fit_transform(data_embed)

# # get all phate embedded states coordinates
phate_all_coords = phate_coords[coord_id_S]

phate_coords.shape, phate_all_coords.shape

In [ ]:
(np.unique(phate_coords,axis=0)).shape, (np.unique(phate_all_coords,axis=0)).shape

### 3. UMAP

In [ ]:
# UMAP set
umap_2d = UMAP(n_components=2, init='random', random_state=0)
# umap_3d = UMAP(n_components=3, init='random', random_state=0)

# UMAP 2D fit tranform
umap_coord_2d = umap_2d.fit_transform(data_embed)
umap_all_coord_2d = umap_coord_2d[coord_id_S]  

# UMAP 3D fit tranform
# umap_coord_3d = umap_3d.fit_transform(data_embed)
# umap_all_coord_3d = umap_coord_3d[coord_id_S]

print((np.unique(umap_coord_2d,axis=0)).shape, (np.unique(umap_coord_3d,axis=0)).shape)
print(umap_all_coord_2d.shape, (np.unique(umap_all_coord_2d,axis=0)).shape)
# print(umap_all_coord_3d.shape, (np.unique(umap_all_coord_3d,axis=0)).shape)

### 4. t-SNE

In [ ]:
# tsne set
tsne_2d = TSNE(n_components=2, perplexity=1000.0, random_state=0)
# tsne_3d = TSNE(n_components=3, random_state=0)

# tsne 2D fit tranform
tsne_coord_2d = tsne_2d.fit_transform(data_embed)
tsne_all_coord_2d = tsne_coord_2d[coord_id_S] 

# # tsne 3D fit tranform
# tsne_coord_3d = tsne_3d.fit_transform(data_embed)
# tsne_all_coord_3d = tsne_coord_3d[coord_id_S] 

 
print((np.unique(tsne_coord_2d,axis=0)).shape, (np.unique(tsne_coord_3d,axis=0)).shape)
print(tsne_all_coord_2d.shape, (np.unique(tsne_all_coord_2d,axis=0)).shape)
# print(tsne_all_coord_3d.shape, (np.unique(tsne_all_coord_3d,axis=0)).shape)

### Save all dats to npz-py and h5-jl

In [ ]:
# """ Save all obtained data to npz file for python,
#     Multiple trajectories
# """
# # save for python
# fnpz_data = "data/helix_assoc/helix_assoc_{}_multrj_30epoch_py.npz".format(SEQ)
# with open(fnpz_data, 'wb') as f:
#     np.savez(f,
#             # SIMS data
#             SIMS_T=SIMS_T, SIMS_HT=SIMS_HT, SIMS_HT_uniq=SIMS_HT_uniq,
#             SIMS_adj_uniq=SIMS_adj_uniq, SIMS_scar_uniq=SIMS_scar_uniq,
#             SIMS_G_uniq=SIMS_G_uniq, SIMS_pair_uniq=SIMS_pair_uniq,
#             SIMS_dict=SIMS_dict, SIMS_dict_uniq=SIMS_dict_uniq,
#             # Indices
#             coord_id_S=coord_id_S, indices_S=indices_S,trj_id=trj_id,
#             # embed data and occpancy density
#             data_embed=data_embed, occ_density_S=occ_density_S,
#             # plotting data
#             pca_coords=pca_coords, pca_all_coords=pca_all_coords,
#             phate_coords=phate_coords, phate_all_coords=phate_all_coords,
#             umap_coord_2d=umap_coord_2d, umap_all_coord_2d=umap_all_coord_2d,
#             umap_coord_3d=umap_coord_3d, umap_all_coord_3d=umap_all_coord_3d,
#             tsne_coord_2d=tsne_coord_2d, tsne_all_coord_2d=tsne_all_coord_2d,
#             tsne_coord_3d=tsne_coord_3d, tsne_all_coord_3d=tsne_all_coord_3d,
#             )

In [ ]:
# """ Save plotting data to h5 file for julia,
#     Multiple trajectories
# """
# fh5_data = "data/helix_assoc/helix_assoc_{}_multrj_30epoch_jl.h5".format(SEQ)
# save_h5(fh5_data,
#         SIMS_G_uniq, SIMS_pair_uniq, occ_density_S, 
#         pca_coords, pca_all_coords,
#         phate_coords, phate_all_coords,
#         umap_coord_2d, umap_all_coord_2d,
#         umap_coord_3d, umap_all_coord_3d,
#         tsne_coord_2d, tsne_all_coord_2d,
#         tsne_coord_3d, tsne_all_coord_3d,
#         )


## 5. Visualize

In [ ]:
SEQ

### 1. PCA Vis

In [ ]:
# data_embed ##before shuflle

In [ ]:
# from sklearn.utils import shuffle

# data_embed = shuffle(data_embed, random_state=0)
# data_embed

In [ ]:
%matplotlib inline
X = pca_all_coords[:,0]
Y = pca_all_coords[:,1]
Z = pca_all_coords[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G,
          cmap='plasma',
          s=20
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]*0.95),fontsize=15,c="yellow", horizontalalignment='center')

In [ ]:
%matplotlib inline
X = pca_coords[:,0]
Y = pca_coords[:,1]
Z = pca_coords[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="yellow")

In [ ]:
X = pca_coords[:,0]
Y = pca_coords[:,1]
Z = pca_coords[:,2]

# PCA: 3 components
fig,ax = plt.subplots(figsize=(8,6))
ax = plt.axes(projection ="3d")

im = ax.scatter3D(X,Y,Z,
          c=SIM_G_uniq,      
          cmap='plasma')
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
z = [Z[0], Z[-1]]
ax.scatter(x,y,z,s=100,c="green",alpha=1)

In [ ]:
X = pca_coords[:,0]
Y = pca_coords[:,1]
Z = pca_coords[:,2]


# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y,
          c=SIMS_pair_uniq,
          cmap='plasma',
          s=15
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="yellow")

#### Try use PCA directly without AE

In [ ]:
pca_coords1 = PCA(n_components=3).fit_transform(SIMS_scar_uniq)   # multiple trj

X = pca_coords1[:,0]
Y = pca_coords1[:,1]
Z = pca_coords1[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="black")

In [ ]:
cm = PCA(n_components=25)
cm.fit(data_embed)

PC_values = np.arange(cm.n_components_) + 1
plt.plot(PC_values, np.cumsum(cm.explained_variance_ratio_), 'ro-', linewidth=2)
plt.title('Scree Plot: PCA')
plt.xlabel('Number of principal components')
plt.ylabel('Cumulative explained variance');
# plt.xticks(np.arange(0, data_embed.shape[-1]+1, 1))

plt.show()

In [ ]:
np.cumsum(cm.explained_variance_ratio_)

### 2. PHATE Vis

In [ ]:
X_phate = phate_all_coords[:,0]
Y_phate = phate_all_coords[:,1]

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X_phate,Y_phate,
                c=SIMS_G,   # multiple trj               
                cmap='plasma',
               )

plt.colorbar(im)

annotations=["I","F"]
x = [X_phate[0],X_phate[-1]]
y = [Y_phate[0],Y_phate[-1]]
plt.scatter(x,y,s=50, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=30,c="black")

In [ ]:
X_phate = phate_coords[:,0]
Y_phate = phate_coords[:,1]

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X_phate,Y_phate,
                c=SIMS_G_uniq,                 
                cmap='plasma',
               )

plt.colorbar(im)

annotations=["I","F"]
x = [X_phate[0],X_phate[-1]]
y = [Y_phate[0],Y_phate[-1]]
plt.scatter(x,y,s=50, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=30,c="black")

#### PHATE without AE

In [ ]:
phate_operator = phate.PHATE(n_jobs=-2)
phate1 = phate_operator.fit_transform(SIMS_scar_uniq)   # multiple trj

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(phate1[:,0],
          phate1[:,1],
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [phate1[:,0][0],phate1[:,0][-1]]
y = [phate1[:,1][0],phate1[:,1][-1]]
plt.scatter(x,y,s=50, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=20,c="black")

### 3. UMAP Vis

In [ ]:
X = umap_coord_2d[:,0]
Y = umap_coord_2d[:,1]
cmap = plt.cm.plasma
cmap_r = plt.cm.get_cmap('plasma_r')

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c = SIMS_G_uniq,
          cmap=cmap,
          s=10
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="yellow")

In [ ]:
# directly UMAP 2D
umap_coord_2dscar = umap_2d.fit_transform(SIMS_scar_uniq)

fig_2d = px.scatter(
    umap_coord_2dscar, x=0, y=1,color=SIMS_G_uniq
)
fig_2d.update_traces(marker_size=3)
fig_2d.show()



In [ ]:
fig_2d = px.scatter(
    umap_coord_2d, x=0, y=1,color=SIMS_G_uniq
)
fig_2d.update_traces(marker_size=3)


fig_3d = px.scatter_3d(
    umap_coord_3d, x=0, y=1, z=2,color=SIMS_G_uniq
)

fig_3d.update_traces(marker_size=2)

fig_2d.show()
fig_3d.show()



### 4. t-SNE Vis

In [ ]:
fig_2d = px.scatter(
    tsne_coord_2d, x=0, y=1,color=SIMS_G_uniq,
    hover_data = {"SIMS_G_uniq":SIMS_G_uniq, 
                  "SIMS_HT_uniq":SIMS_HT_uniq,
                  }
)
fig_2d.update_traces(marker_size=3)
fig_2d.show()


# fig_3d = px.scatter_3d(
#     tsne_coord_3d, x=0, y=1, z=2,color=SIMS_G_uniq
# )
# fig_3d.update_traces(marker_size=2)

